In [ ]:
R.version

In [ ]:
library(qiime2R)
library(tidyr)
library(tibble)
library(ggplot2)
library(vegan)
library(microbiome)
library(phyloseq)
library(dplyr)
library(PCAtools)
library("pairwiseAdonis")

In [ ]:
# Read in the 16S qza files and clean them up for phyloseq
ASVtable_16S <- read_qza("/Users/stephanie.rosales/Documents/Tissueloss/SCTLD_MetaAnalysis/QiimeOutPut/2022_Process/tableV_BacArc_99_SCTLD.qza")
ASVtable_16S <- ASVtable_16S$data # Extract the count data from list
ASVtaxa_16S <- read_qza("/Users/stephanie.rosales/Documents/Tissueloss/SCTLD_MetaAnalysis/QiimeOutPut/2022_Process/taxaVsearch_rep-seqs-dn-99_SCTLD.qza")
taxtable_16S <- ASVtaxa_16S$data %>% as_tibble() %>% separate(Taxon, sep=";",
c("Kingdom", "Phylum", "Class", "Order", "Family", "Genus", "Species")) 
# Convert taxonomy info to data frame with correct taxonomy labels
taxtable_16S <- taxtable_16S[-1,]
#asv_tab$asv_id <- rownames(asv_tab) # add a new column for ids




In [ ]:
sample_info_tab_16S <- read.csv("/Users/stephanie.rosales/Documents/Tissueloss/SCTLD_MetaAnalysis/metadata_for_qiime/SCTLD_meta_analysis_metadata.csv", header = T, row.names = 1,
na.strings = c("", "NA"))

In [ ]:
physeq_16S <- phyloseq(otu_table(ASVtable_16S, taxa_are_rows= T),
tax_table(as.data.frame(taxtable_16S) %>% column_to_rownames("Feature.ID") %>%
as.matrix()), sample_data(sample_info_tab_16S))
physeq_16S
#taxa_names(physeq_16S) <- paste0("ASV", seq(ntaxa(physeq_16S)))

In [ ]:
sub_samples = c("TissueSlurry", "Mucus", "TissueSlurry_Skeleton", "Seawater", "Sediment")

sub_samples2 = c("TissueSlurry", "Mucus", "TissueSlurry_Skeleton")

ps.coral =subset_samples(physeq_16S, sample_type %in% sub_samples & Alias!="AcroporaDisease"
                       )

ps.coral
    

In [ ]:
ps.coral = filter_taxa(ps.coral, function(x) sum(x > 5) > (0.20*length(x)), TRUE)
ps.coral
  

In [ ]:
ps.coral_clr <- microbiome::transform(ps.coral, 'clr')


### Biome

In [ ]:
#dis_clr <- vegdist(otu_table(t(ps.coral)), method ="euclidean")
# PERMDISP2 procedure for the analysis of multivariate homogeneity of group dispersions (variances).
mod_clr <- betadisper(dis_clr, sample_data(ps.coral)$Biome)

boxplot(mod_clr)
permutest(mod_clr, permutations = how(nperm=999))

adonis2(dis_clr~Biome, 
       data =ps_clr_meta, permutations = 999, 
         method = "euclidean", block=Study)




### Primers

In [ ]:
# PERMDISP2 procedure for the analysis of multivariate homogeneity of group dispersions (variances).
mod_clr <- betadisper(dis_clr, sample_data(ps.coral)$primer_names)
permutest(mod_clr, permutations = how(nperm=999))
boxplot(mod_clr)

adonis2(dis_clr~primer_names, 
       data =ps_clr_meta, permutations = 999, 
         method = "euclidean", block=Study)

### Species

In [ ]:
mod_clr <- betadisper(dis_clr, sample_data(ps.coral)$species_code)

boxplot(mod_clr)

permutest(mod_clr, permutations = how(nperm=999))


adonis2(dis_clr~species_code, 
       data =ps_clr_meta, permutations = 999, 
         method = "euclidean", block=Study)



### Sample type

In [ ]:
mod_clr <- betadisper(dis_clr, sample_data(ps.coral)$sample_type)

boxplot(mod_clr)
permutest(mod_clr, permutations = how(nperm=999))




adonis2(dis_clr~sample_type, 
       data =ps_clr_meta, permutations = 999, 
         method = "euclidean", block=Study)



### Study

In [ ]:
mod_clr <- betadisper(dis_clr, sample_data(ps.coral)$Alias)
boxplot(mod_clr)
permutest(mod_clr, permutations = how(nperm=999))


adonis2(dis_clr~Alias, 
       data =ps_clr_meta, permutations = 999, 
         method = "euclidean")


### Year

In [ ]:
mod_clr <- betadisper(dis_clr, sample_data(ps.coral)$collection_year)

boxplot(mod_clr)


permutest(mod_clr, permutations = how(nperm=999))


adonis2(dis_clr~collection_year, 
       data =ps_clr_meta, permutations = 999, 
         method = "euclidean", block=Study)


